In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import tensorflow_privacy
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow_privacy.privacy.analysis import compute_dp_sgd_privacy
from tensorflow.keras.regularizers import L1,L2
from tensorflow.keras import layers
from keras import backend as K
tf.random.set_seed(42)

In [2]:
train, test = tf.keras.datasets.mnist.load_data()
train_data, train_labels = train
test_data, test_labels = test

train_data = np.array(train_data, dtype=np.float32)/255
test_data = np.array(test_data, dtype=np.float32)/255

train_data = train_data.reshape(train_data.shape[0],28,28,1) # angka 1 dibelakang menunjukkan color channel. Hanya ada 1 color channel
test_data = test_data.reshape(test_data.shape[0],28,28,1)

train_labels = np.array(train_labels, dtype = np.int32)
test_labels = np.array(test_labels, dtype = np.int32)

train_labels = tf.keras.utils.to_categorical(train_labels, num_classes = 10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes = 10)

#check data distribution, normalized since divided by 255
assert train_data.min() == 0
assert train_data.max() == 1
assert test_data.min() == 0
assert test_data.max() == 1

## Defining Functions

In [12]:
def dp_sgd(l2_norm_clip,noise_multiplier,num_microbatches,learning_rate):
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(16,8,
                              strides = 2,
                              padding = 'same',
                              activation = 'relu',
                              input_shape = (28,28,1)),
        tf.keras.layers.MaxPool2D(2,1),
        tf.keras.layers.Conv2D(32,4,
                               strides = 2,
                               padding = 'valid',
                               activation = 'relu'),
        tf.keras.layers.MaxPool2D(2,1),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(32, activation = 'relu'),
        tf.keras.layers.Dense(10, activation = 'softmax')
    ])
    
    optimizer = tensorflow_privacy.DPKerasSGDOptimizer(
        l2_norm_clip = l2_norm_clip,
        noise_multiplier = noise_multiplier,
        num_microbatches = num_microbatches,
        learning_rate = learning_rate)
    loss = tf.keras.losses.CategoricalCrossentropy(
        from_logits = False, reduction = tf.losses.Reduction.NONE)
    model.compile(optimizer = optimizer , loss = loss, metrics = ['accuracy'])
    return model

def getlayeroutput(model,data,index):
    from keras import backend as K
    getoutput = K.function([model.layers[0].input],[model.layers[index].output])
    return getoutput([data])[0]

def discriminator(in_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(in_shape*20,input_dim = in_shape,
                              activation = 'relu'),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(1,activation = 'sigmoid')
    ])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = 0.001,
    decay_steps = 200,
    decay_rate = 0.96,
    staircase = True)
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer,
                 metrics = 'accuracy')
    return model

def discriminator(in_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(in_shape*20,input_dim = in_shape,
                              activation = 'relu'),
        tf.keras.layers.Dropout(0.6),
        tf.keras.layers.Dense(1,activation = 'sigmoid')
    ])
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = 0.001,
    decay_steps = 200,
    decay_rate = 0.96,
    staircase = True)
    optimizer = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer,
                 metrics = 'accuracy')
    return model

def discriminatortrue(in_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(in_shape*1.25,input_dim = in_shape,
                              activation = 'linear'),
        tf.keras.layers.Dense(in_shape*2,activation = 'relu'),
        tf.keras.layers.Dense(in_shape*1.5,activation = 'linear'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(in_shape*2,activation = 'relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1,activation = 'sigmoid')
    ])
    optimizer = tf.keras.optimizers.Adam()
    model.compile(loss = 'binary_crossentropy', optimizer = optimizer,
                 metrics = 'accuracy')
    return model

def generator(in_shape):
    model = tf.keras.Sequential([
        #tf.keras.layers.Dense(16,input_dim = in_shape),
        tf.keras.layers.Conv1D(10,2,activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Conv1D(20,2,activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Conv1D(30,2,activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(25,activation='linear'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(10,activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(in_shape,activation='relu'),
    ])
    return model
    
def GAN(gen,dis):
    dis.trainable = False
    model = tf.keras.Sequential([ 
        gen,
        dis
    ])
    opt = tf.keras.optimizers.Adam()
    model.compile(loss='binary_crossentropy',optimizer=opt)
    
    return model


def generate_fake_samples(gen, data, batch):
    x_input = np.random.randn(batch,data.shape[1])
    x_input = np.expand_dims(x_input, axis=2)
    x = gen.predict(x_input)
    y = np.zeros((batch,1))
    
    return x.reshape(batch,data.shape[1]),y

def shuffledata(xdata,ydata):
    indices = tf.range(start=0,limit=tf.shape(xdata)[0], dtype = tf.int32)
    idx = tf.random.shuffle(indices)
    return tf.gather(xdata,idx) , tf.gather(ydata,idx)

def train(gen, dis, distrue, gan, output_noise, output_true, n_epochs= 10000, n_batch = 512):
    #determine half the size of one batch, for updating the discriminator
    half_batch = int(n_batch/2)
    dishist=[]
    genhist=[]
    disacchist = []
    distrueacchist = []
    
    ### HUGE NOTE: Normalmodel is 1, DPmodel is 0
    
    #train the true discriminator to the DPmodel and Normalmodel data
    distrue_output_true_train_x = output_true[:] #setting x data from normalmodel for true discriminant
    distrue_output_true_train_y = np.ones((output_true.shape[0],1)) #setting y data from normalmodel for true discriminant
    
    distrue_output_noise_train_x = output_noise[:] #setting x data from DPmodel for true discriminant
    distrue_output_noise_train_y = np.zeros((output_noise.shape[0],1)) #setting y data from DPmodel for true discriminant
    
    #combine
    distruex = pd.DataFrame(distrue_output_true_train_x).append(pd.DataFrame(distrue_output_noise_train_x),ignore_index = True).to_numpy()
    distruey = pd.DataFrame(distrue_output_true_train_y).append(pd.DataFrame(distrue_output_noise_train_y),ignore_index = True).to_numpy()
    
    #shuffle
    distruex_shuffle, distruey_shuffle = shuffledata(distruex,distruey)
    distrue.fit(distruex_shuffle,distruey_shuffle)
    
    #manually enumerate epochs
    for epoch in range(n_epochs):
        ###########################################################################################################      
        ##GAN PART
        #prepare samples      
        #Normalmodel data
        x_real = output_true[np.random.choice(output_true.shape[0], half_batch, replace=False), :]
        y_real = np.ones((half_batch,1))
        
        #Normalmodel data upsampled (fake)
        x_fake, y_fake = generate_fake_samples(gen, output_true, half_batch)
        
        #combined
        disx = pd.DataFrame(x_real).append(pd.DataFrame(x_fake),ignore_index = True).to_numpy()
        disy = pd.DataFrame(y_real).append(pd.DataFrame(y_fake),ignore_index = True).to_numpy()
        
        #shuffle
        disx_shuffle, disy_shuffle = shuffledata(disx,disy)
        
        #update discriminator
        disloss, disacc = dis.train_on_batch(disx_shuffle,disy_shuffle)
        
        #input for gan
        noise = np.random.randn(n_batch,output_true.shape[1])
        noise = np.expand_dims(noise, axis=2)
        y_gen = np.ones((n_batch,1))
        
        #update generator from discriminator error
        gen_loss_fake = gan.train_on_batch(noise,y_gen)
        
        ###########################################################################################################
        ##TRUE DISCRIMINANT PART
        #Check the true discriminator performance
        #Normalmodel data upsampled (fake)
        distrue_output_true_x = x_fake[:]
        distrue_output_true_y = y_fake[:]
        distrue_output_true_y[:] = 1
    
        #DPmodel data
        distrue_output_noise_x = output_noise[np.random.choice(output_noise.shape[0], half_batch, replace=False), :]
        distrue_output_noise_y = np.zeros((distrue_output_true_y.shape[0],0))
        
        #aggregated
        distruex_test = pd.DataFrame(distrue_output_true_x).append(pd.DataFrame(distrue_output_noise_x),ignore_index = True).to_numpy()
        distruey_test = pd.DataFrame(distrue_output_true_y).append(pd.DataFrame(distrue_output_noise_y),ignore_index = True).to_numpy()
        
        
        distrueloss, distrueacc = distrue.test_on_batch(distruex_test,distruey_test)
        
        print('>%d, discriminant_loss=%.3f discriminant_acc=%.3f generator_loss=%.3f true_discriminant_acc=%.3f' % (epoch+1, disloss, disacc, gen_loss_fake, distrueacc))
        
        dishist.append(disloss)
        genhist.append(gen_loss_fake)
        disacchist.append(disacc)
        distrueacchist.append(distrueacc)
    plot_history(dishist,genhist,disacchist,distrueacchist)
    
    #final accuracy of True Discriminant
    output_true_upsampled_x, output_true_upsampled_y = generate_fake_samples(gen, output_true, output_true.shape[0])
    output_true_upsampled_y[:] = 1
    
    output_noise_x, output_noise_y = output_noise[:], np.zeros((output_noise.shape[0],1))
    
    distruex_final = pd.DataFrame(output_true_upsampled_x).append(pd.DataFrame(output_noise_x),ignore_index = True).to_numpy()
    distruey_final = pd.DataFrame(output_true_upsampled_y).append(pd.DataFrame(output_noise_y),ignore_index = True).to_numpy()
    
    #shuffle
    distruex_final_shuffle, distruey_final_shuffle = shuffledata(distruex_final,distruey_final)
    
    fin_loss, fin_acc = distrue.test_on_batch(distruex_final_shuffle, distruey_final_shuffle)
    
    print("Final True Discriminant Accuracy: %.3f" % fin_acc)
        
        
def plot_history(d_hist, g_hist, dacc_hist, dacc2_hist):
    # plot loss    
    plt.figure(1)
    plt.subplot(211)
    plt.plot(d_hist, label='discriminator_loss')
    plt.plot(g_hist, label='generator_loss')
    plt.legend()
    plt.xlabel('Epochs')
    plt.subplot(212)
    plt.plot(dacc_hist, label='GAN_discriminator_accuracy')
    plt.plot(dacc2_hist, label='True_discriminator_accuracy')
    plt.legend()
    plt.xlabel('Epochs')
    plt.show()
    plt.close()

## Training Original Model 

##### DPmodel & Normalmodel

In [4]:
DPmodel = dp_sgd(l2_norm_clip = 1.5, noise_multiplier = 1.8, 
                 num_microbatches = 50, learning_rate = 0.25)
Normalmodel = dp_sgd(l2_norm_clip = 1.5, noise_multiplier = 1, 
                 num_microbatches = 50, learning_rate = 0.25)

DPmodel.fit(train_data, train_labels, 
            epochs = 5, batch_size = 250,
            validation_data = (test_data, test_labels))

Normalmodel.fit(train_data, train_labels, 
                epochs = 5, batch_size = 250,
                validation_data = (test_data, test_labels))

Epoch 1/5
240/240 [==============================] - 39s 132ms/step - loss: 1.1429 - accuracy: 0.6526 - val_loss: 1.2173 - val_accuracy: 0.7374
Epoch 2/5
240/240 [==============================] - 30s 126ms/step - loss: 1.3376 - accuracy: 0.7543 - val_loss: 1.3906 - val_accuracy: 0.7351
Epoch 3/5
240/240 [==============================] - 30s 125ms/step - loss: 1.3964 - accuracy: 0.7087 - val_loss: 1.3229 - val_accuracy: 0.7082
Epoch 4/5
240/240 [==============================] - 30s 125ms/step - loss: 1.4388 - accuracy: 0.6870 - val_loss: 1.2392 - val_accuracy: 0.7103
Epoch 5/5
240/240 [==============================] - 30s 127ms/step - loss: 1.3609 - accuracy: 0.6710 - val_loss: 1.2486 - val_accuracy: 0.6971
Epoch 1/5
240/240 [==============================] - 33s 128ms/step - loss: 0.7731 - accuracy: 0.7581 - val_loss: 0.4271 - val_accuracy: 0.8859
Epoch 2/5
240/240 [==============================] - 30s 126ms/step - loss: 0.4367 - accuracy: 0.8986 - val_loss: 0.4281 - val_accuracy:

##### DP Model Privacy

In [5]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=train_data.shape[0],
                                              batch_size=250,
                                              noise_multiplier=1.6,
                                              epochs=5,
                                              delta=1e-5)

DP-SGD with sampling rate = 0.417% and noise_multiplier = 1.6 iterated over 1200 steps satisfies differential privacy with eps = 0.422 and delta = 1e-05.
The optimal RDP order is 27.0.


(0.4223191547406633, 27.0)

##### Normal Model Privacy

In [6]:
compute_dp_sgd_privacy.compute_dp_sgd_privacy(n=train_data.shape[0],
                                              batch_size=250,
                                              noise_multiplier=1,
                                              epochs=5,
                                              delta=1e-5)

DP-SGD with sampling rate = 0.417% and noise_multiplier = 1 iterated over 1200 steps satisfies differential privacy with eps = 1.13 and delta = 1e-05.
The optimal RDP order is 10.0.


(1.1278495014370558, 10.0)

## Proceed with GAN

In [7]:
output_model_noise = DPmodel.predict(train_data)
output_model_true = Normalmodel.predict(train_data)

1875/1875 [==============================] - 8s 4ms/step


In [13]:
for i in range(0,6):
    disc2 = discriminatortrue(output_model_noise.shape[1])
    disc = discriminator(output_model_true.shape[1])
    gen = generator(output_model_true.shape[1])
    gan = GAN(gen,disc)
    train(gen,disc,disc2,gan,output_model_noise,output_model_true,n_epochs = 1000)

ValueError: Input 0 of layer "conv1d_8" is incompatible with the layer: expected min_ndim=3, found ndim=2. Full shape received: (None, 200)

In [ ]:
noise = np.random.randn(output_model_noise.shape[0],output_model_noise.shape[1])
noise = np.expand_dims(noise, axis=2)
comparenoise = output_model_noise.flatten()
comparetrue = output_model_true.flatten()
comparefake = gen.predict(noise).flatten()
sns.kdeplot(comparenoise)
sns.kdeplot(comparetrue)
plt.legend(labels = ['comparenoise','comparetrue'])
plt.show()
sns.kdeplot(comparefake)
plt.legend(labels = 'comparefake')
plt.show()

In [ ]:
from scipy.stats import ttest_ind

ttest_ind(comparetrue,comparefake)

In [ ]:
sns.kdeplot(comparetrue)
sns.kdeplot(comparefake)
plt.legend(labels = ['comparetrue','comparefake'])